In [1]:
import gammagl.transforms as T
import tensorlayerx as tlx
from gammagl.datasets import Planetoid
from gammagl.layers.conv import GCNConv

Using PyTorch backend.


Gammagl Global Config Info:
    TLX_BACKEND: torch
    DATASET_ROOT: C:\Users\mi\.ggl\datasets


In [37]:
dataset = Planetoid(root="./data", name="Cora", transform=T.NormalizeFeatures())

In [87]:
class GCN(tlx.nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        self.conv1 = GCNConv(in_feats, hid_feats)
        self.conv2 = GCNConv(hid_feats, out_feats)
        self.relu = tlx.ReLU()

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [88]:
model = GCN(dataset.num_features, 16, dataset.num_classes)

[TLX] Linear  linear_16: 16 No Activation
[TLX] Linear  linear_17: 7 No Activation


In [89]:
print(type(dataset))
print(type(dataset[0]))

<class 'gammagl.datasets.planetoid.Planetoid'>
<class 'gammagl.data.graph.Graph'>


In [90]:
data = dataset[0]
data

Graph(edge_index=[2, 10556], x=[2708, 1433], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [92]:
import torch
import torch.nn.functional as F

data = dataset[0]
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(200):
    pred = model(data.x, data.edge_index)
    loss = F.cross_entropy(pred[data.train_mask], data.y[data.train_mask])

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(loss.detach().item())

0.015068531036376953
0.006627872120589018
0.003203995293006301
0.0018103520851582289
0.0011818408966064453
0.0008583818562328815
0.0006686152773909271
0.000544812239240855
0.00045719227637164295
0.0003915383422281593
0.00034033306292258203
0.00029931816970929503
0.00026571890339255333
0.00023778971808496863
0.00021426868624985218
0.0001942425296874717
0.0001770384405972436
0.000162104784976691
0.00014907443255651742
0.00013763805327471346
